# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1. Read the data
tic_df = pd.read_csv('tic-tac-toe.csv')

In [ ]:
tic_df.isnull().sum()

,0
TL,0
TM,0
TR,0
ML,0
MM,0
MR,0
BL,0
BM,0
BR,0
class,0


In [ ]:
# 2. Inspect the dataset
print("Dataset Info:")
print(tic_df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None


In [ ]:
tic_df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [ ]:
tic_df.shape

(958, 10)

In [ ]:
print("\nValue counts for each column:")
for column in tic_df.columns:
    print(tic_df[column].value_counts())


Value counts for each column:
TL
x    418
o    335
b    205
Name: count, dtype: int64
TM
x    378
o    330
b    250
Name: count, dtype: int64
TR
x    418
o    335
b    205
Name: count, dtype: int64
ML
x    378
o    330
b    250
Name: count, dtype: int64
MM
x    458
o    340
b    160
Name: count, dtype: int64
MR
x    378
o    330
b    250
Name: count, dtype: int64
BL
x    418
o    335
b    205
Name: count, dtype: int64
BM
x    378
o    330
b    250
Name: count, dtype: int64
BR
x    418
o    335
b    205
Name: count, dtype: int64
class
True     626
False    332
Name: count, dtype: int64


In [ ]:
# 3. Convert categorical to numeric
# Mapping: x=1, o=0, b=-1 for board positions; True=1, False=0 for class
mapping = {'x': 1, 'o': 0, 'b': -1, True: 1, False: 0}
for column in tic_df.columns:
    tic_df[column] = tic_df[column].map(mapping)

print("\nAfter conversion to numeric:")
tic_df.head()


After conversion to numeric:


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,-1,-1,1
4,1,1,1,1,0,0,-1,0,-1,1


In [ ]:
for col in ['BL', 'BM', 'BR']:
    print(f"Unique values in column '{col}': {tic_df[col].unique()}")

Unique values in column 'BL': [ 1  0 -1]
Unique values in column 'BM': [ 0  1 -1]
Unique values in column 'BR': [ 0  1 -1]


In [ ]:
# 4. Separate inputs and output
X = tic_df.iloc[:, :-1]  # All columns except the last one
y = tic_df.iloc[:, -1]   # Last column (class)

print("\nInput features shape:", X.shape)
print("Output shape:", y.shape)


Input features shape: (958, 9)
Output shape: (958,)


In [ ]:
from sklearn.preprocessing import StandardScaler
# StandardScaler instead of min-max
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Create Sequential model
model = keras.Sequential()
model.add(keras.layers.Dense(64, activation='relu', input_shape=(9,)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#optimizer = keras.optimizers.Adam(learning_rate=0.0001)  # Default is 0.001, but now explicit
#model.compile(
    #optimizer=optimizer,
    #loss='sparse_categorical_crossentropy',
    #metrics=['accuracy']
#)

In [ ]:
# Print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,786 (10.88 KB)

 Trainable params: 2,786 (10.88 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fit the training data
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=8,
    validation_data=(X_test, y_test) # Use validation_data since we have X_test and y_test,
)

Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6462 - loss: 0.6473 - val_accuracy: 0.6667 - val_loss: 0.5957
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7175 - loss: 0.5656 - val_accuracy: 0.7031 - val_loss: 0.5763
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7585 - loss: 0.5288 - val_accuracy: 0.7135 - val_loss: 0.5610
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7852 - loss: 0.4887 - val_accuracy: 0.7292 - val_loss: 0.5432
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7809 - loss: 0.4934 - val_accuracy: 0.7396 - val_loss: 0.5293
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8058 - loss: 0.4429 - val_accuracy: 0.7552 - val_loss: 0.5103
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8130 - loss: 0.4431 - val_accuracy: 0.7604 - val_loss: 0.4909
Epoch 8/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8249 - loss: 0.3984 - val_accuracy: 0.7708 - val_loss:

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Save the model
model.save('tic-tac-toe.keras')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9330 - loss: 0.1634  

Test Accuracy: 0.9323
Test Loss: 0.1747


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# Load the saved model (using the correct filename)
loaded_model = keras.models.load_model("tic-tac-toe.keras")

# Convert y_test to NumPy array if it's a Pandas Series
y_test_array = y_test.to_numpy() if hasattr(y_test, 'to_numpy') else y_test

# Select 5 random samples
random_indices = np.random.choice(len(X_test), 10, replace=False)
X_sample = X_test[random_indices]
y_sample = y_test_array[random_indices]

# Make predictions
predictions = loaded_model.predict(X_sample)
predicted_labels = np.argmax(predictions, axis=1)

# Print results in your requested format
print("Here's what the model predicts compared to reality:")
for i in range(len(X_sample)):
    print(f"Predicted: {predicted_labels[i]}, Actual: {y_sample[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Here's what the model predicts compared to reality:
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
import tensorflow as tf


# Create Sequential model
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation='relu', input_shape=(9,))) # new
model.add(keras.layers.Dense(64, activation='relu', input_shape=(9,)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu', input_shape=(9,))) # new
model.add(keras.layers.Dropout(0.3))  # Single Dropout, as in original
model.add(keras.layers.Dense(2, activation='softmax'))

# Compile with default Adam
#model.compile(
    #optimizer='adam',  # Default learning rate = 0.001
    #loss='sparse_categorical_crossentropy',
    #metrics=['accuracy']
#)

# Compile with slightly lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0007)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Add Early Stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Fit the training data
history = model.fit(
    X_train, y_train,
    epochs=75,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Save the model
model.save('tic-tac-toe-improved.keras')

Epoch 1/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6074 - loss: 0.6705 - val_accuracy: 0.6562 - val_loss: 0.6225
Epoch 2/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6735 - loss: 0.6104 - val_accuracy: 0.6667 - val_loss: 0.6017
Epoch 3/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6930 - loss: 0.5763 - val_accuracy: 0.7500 - val_loss: 0.5678
Epoch 4/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7676 - loss: 0.5344 - val_accuracy: 0.7604 - val_loss: 0.5402
Epoch 5/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8044 - loss: 0.4910 - val_accuracy: 0.7812 - val_loss: 0.5098
Epoch 6/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7853 - loss: 0.4720 - val_accuracy: 0.7812 - val_loss: 0.4837
Epoch 7/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8199 - loss: 0.4296 - val_accuracy: 0.8229 - val_loss: 0.4514
Epoch 8/75
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8821 - loss: 0.3732 - val_accuracy: 0.8229 - val_loss

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# To improve my model’s performance, I found several approaches particularly helpful.
# First, increasing the number of training epochs allowed the model more time to learn
# from the data, which enhanced its ability to fit the training set effectively.
# Second, adding more layers to the model increased its capacity to capture complex
# patterns in the data, contributing to better overall performance. Finally, carefully
# adjusting the learning rate proved crucial—by fine-tuning it, I could balance the
# speed of convergence with the model’s ability to generalize, preventing issues like
# overfitting or underfitting. Together, these strategies—extending epochs, deepening
# the architecture, and controlling the learning rate—significantly boosted the
# model’s accuracy and reduced its loss.